<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2020-04-06 19:41:08--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  33.5MB/s    in 8.0s    

2020-04-06 19:41:17 (31.0 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [0]:
!unzip concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.


Use the following cells to create your classifier.

In [0]:
# import all modules, using tensorflow 1.x

%tensorflow_version 1.x
#ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
#Keras module for net composing
import keras
from keras.models import Sequential
from keras.layers import Dense
#VGG16
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

In [0]:
# model parameters
num_classes = 2
image_resize = 224
batch_size_training = 100
batch_size_validation = 100

In [0]:
# data generator for batching/data augmenting
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [6]:
# train and validation generator
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')


Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [63]:
# model definition w/ summary of parameters
model = Sequential()
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))
model.add(Dense(num_classes, activation='softmax'))
model.layers[0].trainable = False
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [0]:
# epoch steps for training and validation
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [9]:
# model training
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/2
301/301 [==============================] - 203s 675ms/step - loss: 0.0971 - acc: 0.9646 - val_loss: 0.0252 - val_acc: 0.9947
Epoch 2/2
301/301 [==============================] - 192s 639ms/step - loss: 0.0200 - acc: 0.9954 - val_loss: 0.0152 - val_acc: 0.9962


In [0]:
# saving model
model.save('classifier_vgg16_model.h5')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [0]:
# mounting gdrive for resnet model loading
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# importing trained resnet model
from keras.models import load_model
resnet_model = load_model('/content/gdrive/My Drive/Colab Notebooks/classifier_resnet_model.h5')

In [0]:
# re-assign vgg model
vgg_model = model

In [11]:
# test generator for model testing
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    shuffle = False)

Found 500 images belonging to 2 classes.


In [23]:
# resnet model evaluation
resnet_evaluation = resnet_model.evaluate_generator(test_generator, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=1)

16/16 [==============================] - 5s 319ms/step


In [14]:
# vgg model evaluation
vgg_evaluation = vgg_model.evaluate_generator(test_generator, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=1)

16/16 [==============================] - 7s 443ms/step


In [30]:
print("The VGG model performed with a loss of {:.3f}".format(vgg_evaluation[0]))
print("and an accuracy of {:.1%}".format(vgg_evaluation[1]))
print("")
print("The ResNet model performed with a loss of {:.3f}".format(resnet_evaluation[0]))
print("and an accuracy of {:.1%}".format(resnet_evaluation[1]))


The VGG model performed with a loss of 0.012
and an accuracy of 99.8%

The ResNet model performed with a loss of 0.139
and an accuracy of 95.2%


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [33]:
vgg_prediction = vgg_model.predict_generator(test_generator, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=1)

16/16 [==============================] - 4s 224ms/step


In [62]:
print("First five prediction with VGG are:")
for i in vgg_prediction.argmax(axis=-1)[0:5]:
  print(list(test_generator.class_indices.items())[i][0])

First five prediction with VGG are:
negative
negative
negative
negative
negative


In [32]:
resnet_prediction = resnet_model.predict_generator(test_generator, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=1)

16/16 [==============================] - 2s 128ms/step


In [61]:


print("First five prediction with Resnet are:")
for i in resnet_prediction.argmax(axis=-1)[0:5]:
  print(list(test_generator.class_indices.items())[i][0])
  

First five prediction with Resnet are:
negative
negative
negative
negative
negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).